In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
# 1. Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#laod the test dataset MNIST Data:
PATH =  '/content/drive/MyDrive/NeuroTech/Preparation_Sun_Wednesday/FifthSession/'
test_dataset = pickle.load(open(PATH +"test_dataset.pkl","rb"))

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
# load the scripted model
preTrained_model = torch.jit.load(PATH + "traced_model.pt")

In [ ]:
#Good link: https://medium.com/octavian-ai/a-simple-explanation-of-the-inception-score-372dff6a8c7a
def IS(probs):
  #calculate marginal probability p(y)
  p_y = torch.mean(probs, axis = 0)#for each class
  #calculate k_l KL divergence for each outcome
  kl_div = torch.sum( probs * torch.log(probs / p_y), axis = 1)#just for showing
  #calculate IS
  inception_score = torch.exp(kl_div.mean())
  return inception_score


def MNIST_Classifier_Score(model , generated_images , n_splits = 10):
  #this function try to mimic Inception score function to assess the generative models like GAN:
  #to measure diversity and quality of generated MNIST daata
  #so we need to leverage from our classifier to assess these architectures.
  #note our classifier accuray reach to 97.8% , so it may be authentic to do this task!
  ################################
  ###### Prediction Step #########
  ################################
  test_loader = DataLoader(generated_images, batch_size=64)
  y_pred = []
  with torch.no_grad():
      for features, target in test_loader:
        features, labels = features.to(device), target.to(device)
        logits, y_hat = model(features)
        y_pred.extend(y_hat)
  ################################
  ###### Calculate IS ############
  ################################

  print("no of prediction output: ", torch.stack(y_pred).shape)
  print( "no of Split Batches  : ", int(len(y_pred)/n_splits))
  labels_batches = torch.split(torch.stack(y_pred), int(len(y_pred)/n_splits))#divide labels predictions to chunks based on n-splits
  Is_s = np.array([IS(probs).cpu().numpy() for probs in labels_batches])
  return Is_s.mean() , Is_s.std()



In [ ]:
#let's try on test dataset :10,000:
m , s = MNIST_Classifier_Score(model = preTrained_model, generated_images = test_dataset  , n_splits = 10)

no of prediction output:  torch.Size([10000, 10])
no of Split Batches  :  1000


In [ ]:
m,s

(np.float32(9.713059), np.float32(0.09543667))